In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Notebook local path should be at `ScientificValueAgent/figures`.

In [ ]:
SAVE_FIGURES = False

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from itertools import product

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from tqdm import tqdm

In [ ]:
from sva import utils

Set some plotting defaults.

In [ ]:
utils.set_defaults()

# Two-phase sine result

In [ ]:
from sva.postprocessing import read_data, parse_results_by_acquisition_function
from sva.truth.sine2phase import (
    get_phase_plot_info,
    phase_1_sine_on_2d_raster,
    points_in_10_percent_range,
    sine2phase_interpolant_2d,
    sine2phase_residual_2d_phase_relative_mae,
    sine2phase_residual_2d_phase_mse,
    sine2phase_compute_metrics_all_acquisition_functions_and_LTB,
)
from sva.truth.common import limited_time_budget

Get the results. This is around 4 GB of raw data and thus could take a little time to load.

In [ ]:
results = read_data("results/results_22-12-21_sine2phase")

In [ ]:
results_by_acqf = parse_results_by_acquisition_function(results)

## Core manuscript figure

In [ ]:
x, y, Z = get_phase_plot_info(phase_1_sine_on_2d_raster)
X, Y = np.meshgrid(x, y)
gradZ = np.array(np.gradient(Z))
gradZ = np.sqrt((gradZ**2).sum(axis=0))

In [ ]:
extent = (x[0], x[-1], y[0], y[-1])
scale = 1
lw = 0.5

g = np.linspace(0, 1, 100)

### Subfigure (a)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2*scale, 2*scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=mpl.cm.seismic, extent=extent, alpha=0.4)

ax.text(0.05, 0.95, r"(a)", ha="left", va="top", transform=ax.transAxes)
ax.text(0.95, 0.95, "Phase 1", ha="right", va="top", transform=ax.transAxes)
ax.text(0.05, 0.05, "Phase 2", ha="left", va="bottom", transform=ax.transAxes)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

utils.set_grids(ax, grid=False)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

if SAVE_FIGURES:
    plt.savefig("sine2phase_subfigure_a.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

### Subfigure $p(x)$ colorbar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2*scale, 2*scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=mpl.cm.seismic, extent=extent, alpha=0.4)

cbar = utils.add_colorbar(im, aspect=20)
cbar.set_ticks([0, 1])
cbar.set_label(r"$p(\mathbf{x})$", labelpad=-10)

ax.remove()

if SAVE_FIGURES:
    plt.savefig("sine2phase_p_cbar.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

### Subfigure (b)

In [ ]:
single_result = results_by_acqf["UpperConfidenceBound10"][0]
Z = single_result._record[-1]["mu"].reshape(100, 100).T
Z = Z - Z.min()
Z = Z / Z.max()
cmap = mpl.cm.magma

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2*scale, 2*scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)

sine = 0.5 + np.sin(2.0 * np.pi * g) / 4
sine_upper = sine + 0.05
sine_lower = sine - 0.05

# ax.fill_between(g, sine_lower, sine_upper, hatch="|"*10, color=None, alpha=0, linewidth=0)
color="b"
ax.plot(g, sine, f"{color}-", linewidth=lw)
ax.plot(g, sine_upper, f"{color}--", linewidth=lw)
ax.plot(g, sine_lower, f"{color}--", linewidth=lw)

X = single_result.data.X
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5, zorder=3)

ax.text(0.05, 0.95, r"(b)", ha="left", va="top", transform=ax.transAxes)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])

utils.set_grids(ax, grid=False)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

if SAVE_FIGURES:
    plt.savefig("sine2phase_subfigure_b.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

### Subfigure value colorbar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2*scale, 2*scale), sharex=True, sharey=True)

im = ax.imshow(Z, interpolation='bilinear', origin='lower', cmap=cmap, extent=extent, alpha=0.4)

cbar = utils.add_colorbar(im, aspect=20)
cbar.set_ticks([0, 1])
cbar.set_label(r"$U(\mathbf{x}, X)$", labelpad=-10)

ax.remove()

if SAVE_FIGURES:
    plt.savefig("sine2phase_U_cbar.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

### Subfigure (c): select metrics

In [ ]:
acquisition_function_name_maps = {
    "Linear": "LTB",
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)"
}

In [ ]:
all_metrics_linear = sine2phase_compute_metrics_all_acquisition_functions_and_LTB(
    results_by_acqf,
    metric_kwargs={
        "grid_points": 100,
        "interpolation_method": "linear",
    },
)

In [ ]:
all_metrics_nearest = sine2phase_compute_metrics_all_acquisition_functions_and_LTB(
    results_by_acqf,
    metric_kwargs={
        "grid_points": 100,
        "interpolation_method": "nearest",
    },
)

In [ ]:
all_metrics_cubic = sine2phase_compute_metrics_all_acquisition_functions_and_LTB(
    results_by_acqf,
    metric_kwargs={
        "grid_points": 100,
        "interpolation_method": "cubic",
    },
)

In [ ]:
only_plot = ["LTB", "EI", "UCB(10)"]
metric_dictionarys = {"nearest": all_metrics_nearest, "linear": all_metrics_linear, "cubic": all_metrics_cubic}

In [ ]:
n_plots = 3
fig, axs = plt.subplots(1, n_plots, figsize=(1.5 * n_plots, 2), sharex=True, sharey=True)

for ii, (metric_method_name, metric_dictionary) in enumerate(metric_dictionarys.items()):
    
    ax = axs[ii]
    ax.set_title(metric_method_name)
    
    for acquisition_function_name in acquisition_function_name_maps.keys():

        values = metric_dictionary["metrics"][acquisition_function_name]
        metrics_grid = metric_dictionary["metrics_grid"]
        metrics_grid_linear = metric_dictionary["metrics_grid_linear"]
        label = acquisition_function_name_maps[acquisition_function_name]

        if only_plot is None or label in only_plot:
            mu = values.mean(axis=1)
            sd = values.std(axis=1) / 3.0
            if label == "LTB":
                ax.plot(metrics_grid_linear, mu, label=label)
                ax.fill_between(metrics_grid_linear, mu - sd, mu + sd, linewidth=0, alpha=0.3)
            else:
                ax.plot(metrics_grid, mu, label=label)
                ax.fill_between(metrics_grid, mu - sd, mu + sd, linewidth=0, alpha=0.3)


for ax in axs.flatten():
    utils.set_grids(ax)
    ax.tick_params(which="minor", bottom=False, top=False)
    ax.set_xticks([3, 50, 150, 250])
    ax.set_yscale('log')

# ax.axhline(0.1, color="black", linewidth=0.5, linestyle="--", zorder=-1)

ax.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")
ax.text(0.05, 0.05, r"$\mu \pm \sigma / 3$", ha="left", va="bottom", transform=ax.transAxes)
axs[0].text(0.95, 0.95, r"(a)", ha="right", va="top", transform=ax.transAxes)
axs[1].text(0.95, 0.95, r"(b)", ha="right", va="top", transform=ax.transAxes)
axs[2].text(0.95, 0.95, r"(c)", ha="right", va="top", transform=ax.transAxes)

axs[1].set_xlabel(r"$N$")
axs[0].set_ylabel("MSE")

if SAVE_FIGURES:
    plt.savefig("sine2phase_subfigure_c.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

## Supplementary information

### Plot the 2-dimensional interpolation

This is mainly for debugging the 2d interpolation metric which will be used in this example and `xrd4phase` (aka `xrd2dim`).

In [ ]:
interpolation_method="linear"

In [ ]:
single_result = results_by_acqf["ExpectedImprovement"][3]
X = single_result.data.X[:144, :]
true_phase_prop, interpolated_phase_prop = sine2phase_interpolant_2d(X, grid_points=200, interpolation_method=interpolation_method)

In [ ]:
X_LTB = limited_time_budget(12, 2)
_, interpolated_phase_prop_LTB = sine2phase_interpolant_2d(X_LTB, grid_points=200, interpolation_method=interpolation_method)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(6*scale, 2*scale), sharex=True, sharey=True)

ax = axs[0]

im = ax.imshow(true_phase_prop, interpolation='bilinear', origin='lower', cmap=mpl.cm.PuOr, extent=extent, alpha=0.8)

ax.text(0.05, 0.95, r"(a)", ha="left", va="top", transform=ax.transAxes, color="white")
ax.text(0.95, 0.95, "Phase 1", ha="right", va="top", transform=ax.transAxes, color="white")
ax.text(0.05, 0.05, "Phase 2", ha="left", va="bottom", transform=ax.transAxes)
ax.set_title("Dense Grid")

ax = axs[1]
im = ax.imshow(interpolated_phase_prop - true_phase_prop, interpolation='bilinear', origin='lower', cmap=mpl.cm.seismic, extent=extent, alpha=0.4, vmin=-0.2, vmax=0.2)
ax.scatter(X[:, 0], X[:, 1], s=0.3, color='black', alpha=1)
ax.text(0.05, 0.95, r"(b)", ha="left", va="top", transform=ax.transAxes)
ax.set_title("AQF: EI")
# utils.add_colorbar(im)

ax = axs[2]
im = ax.imshow(interpolated_phase_prop_LTB - true_phase_prop, interpolation='bilinear', origin='lower', cmap=mpl.cm.seismic, extent=extent, alpha=0.4, vmin=-0.2, vmax=0.2)
ax.scatter(X_LTB[:, 0], X_LTB[:, 1], s=0.3, color='black', alpha=1)
ax.text(0.05, 0.95, r"(c)", ha="left", va="top", transform=ax.transAxes)
ax.set_title("LTB")
# utils.add_colorbar(im)


ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
for ax in axs:
    utils.set_grids(ax, grid=False)

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylabel("$x_2$~[a.u.]", labelpad=10)
ax.set_xlabel("$x_1$~[a.u.]", labelpad=10)

if SAVE_FIGURES:
    plt.savefig("sine2phase_compared_with_interpolation_SI.svg", dpi=300, bbox_inches="tight")
else:
    plt.show()

### Multi-panel densities

In [ ]:
acquisition_function_name_maps = {
    "ExpectedImprovement": "EI",
    "UpperConfidenceBound1": "UCB(1)",
    "UpperConfidenceBound10": "UCB(10)",
    "UpperConfidenceBound20": "UCB(20)",
    "UpperConfidenceBound100": "UCB(100)"
}

In [ ]:
all_points = dict()
for acquisition_function_name, values in results_by_acqf.items():
    tmp_metrics = [exp.data.X for exp in values]
    all_points[acquisition_function_name] = np.array(tmp_metrics)

In [ ]:
N_to_plot = [3, 50, 100, 150, 200, 250]

In [ ]:
L1 = len(acquisition_function_name_maps)
L2 = len(N_to_plot)
fig, axs = plt.subplots(L1, L2, figsize=(L2, L1), sharex=True, sharey=True)

for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    value = all_points[acquisition_function_name]
    label = acquisition_function_name_maps[acquisition_function_name]
    for jj, N in enumerate(N_to_plot):
        ax = axs[ii, jj]
        X = value[:, :N, :].reshape(-1, 2)
        ax.hist2d(X[:, 0], X[:, 1], bins=50, cmap="RdBu_r")

for jj, N in enumerate(N_to_plot):
    ax = axs[0, jj]
    ax.set_title(r"$N=%i$" % N)
for ii, acquisition_function_name in enumerate(acquisition_function_name_maps.keys()):
    label = acquisition_function_name_maps[acquisition_function_name]
    ax = axs[ii, -1]
    ax.text(1.05, 0.5, label, ha="left", va="center", transform=ax.transAxes, rotation=90)
for ax in axs.flatten():
    ax.set_axis_off()

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
# Use `labelpad=...` for padding when using ax.set_xlabel(...)
ax.set_ylabel("$x_2$~[a.u.]")
ax.set_xlabel("$x_1$~[a.u.]")

plt.subplots_adjust(hspace=0.03, wspace=0.03)

if SAVE_FIGURES:
    plt.savefig("sine2phase_SI_hist.pdf", dpi=300, bbox_inches="tight")
else:
    plt.show()